In [2]:
# %pip install llama-index-llms-openai
# %pip install llama-index-readers-file pymupdf

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [4]:
from pathlib import Path

# Create data directory safely (Windows / Linux / macOS)
DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)


### Load Data

In [5]:
from pathlib import Path

from llama_index.readers.file import PDFReader
from llama_index.readers.file import PyMuPDFReader

In [6]:
loader = PyMuPDFReader()
# docs0 = loader.load_data(file=Path("./data/llama2.pdf"))
docs0 = loader.load(file_path=Path("./data/GMDSS_System-IOM_Manual.pdf"))

By default, the PDF reader creates a separate doc for each page. For the sake of this notebook, we stitch docs together into one doc. This will help us better highlight auto-merging capabilities that “stitch” chunks together later on.

In [7]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

### Parse Chunk Hierarchy from Text, Load into Storage

By default, the hierarchy is:

- 1st level: chunk size 2048
- 2nd level: chunk size 512
- 3rd level: chunk size 128

In [ ]:
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import SentenceSplitter

In [9]:
node_parser = HierarchicalNodeParser.from_defaults()

In [10]:
nodes = node_parser.get_nodes_from_documents(docs)

In [11]:
len(nodes)

1219

Here we import a simple helper function for fetching “leaf” nodes within a node list. These are nodes that don’t have children of their own.

In [12]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

In [13]:
leaf_nodes = get_leaf_nodes(nodes)

In [14]:
len(leaf_nodes)

934

In [15]:
root_nodes = get_root_nodes(nodes)

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
import chromadb

# Setup
embed_model = AzureOpenAIEmbedding(model="text-embedding-3-large")
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection("rag_hier")
vector_store = ChromaVectorStore(chroma_collection=collection)

# Load multiple PDFs
reader = SimpleDirectoryReader("./pdfs_dir")
docs = reader.load_data()

# Hierarchical parse
node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=[2048, 512, 128])
nodes = node_parser.get_nodes_from_documents(docs)
leaf_nodes = get_leaf_nodes(nodes)  # Import from llama_index.core.node_parser.relational.hierarchical

# Optional: Add parent summary (cost-optimized loop)
for leaf in leaf_nodes:
    if leaf.relationships.PARENT:
        parent = next(n for n in nodes if n.node_id == leaf.relationships.PARENT.node_id)
        summary = gpt_4o_mini_client.chat.completions.create(prompt=f"Summarize: {parent.text}")  # Truncated
        leaf.text = f"Parent hierarchy: {summary.choices[0].message.content}\n\n{leaf.text}"

# Embed and store leaves
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(leaf_nodes, embed_model=embed_model, storage_context=storage_context)
